In [1]:
import os.path
import torch

PATH_TO_PRETRAINED = '/home/vika/cqas_flask/external_pretrained_models/'
MODEL_NAMES = ['bert_simple1.hdf5']

def load(checkpoint_fn, gpu=-1):
    if not os.path.isfile(PATH_TO_PRETRAINED + checkpoint_fn):
        raise ValueError('Can''t find tagger in file "%s". Please, run the main script with non-empty \
                         "--save-best-path" param to create it.' % checkpoint_fn)
    tagger = torch.load(PATH_TO_PRETRAINED + checkpoint_fn)
    tagger.gpu = gpu

    tagger.word_seq_indexer.gpu = gpu # hotfix
    tagger.tag_seq_indexer.gpu = gpu # hotfix
    if hasattr(tagger, 'char_embeddings_layer'):# very hot hotfix
        tagger.char_embeddings_layer.char_seq_indexer.gpu = gpu # hotfix
    tagger.self_ensure_gpu()
    return tagger

In [2]:
!pwd

/home/vika/cqas_flask


In [3]:
! ls /home/vika/NER_RNN/targer/src/layers

ls: cannot access '/home/vika/NER_RNN/targer/src/layers': No such file or directory


In [4]:
import torch 
print(torch.__version__)

1.3.1


In [5]:
import sys
sys.path.append('/home/vika/targer')
from src.factories.factory_tagger import TaggerFactory
from src.layers import layer_context_word_embeddings_bert

for MODEL_NAME in ['bert_simple1.hdf5']:
    print (MODEL_NAME)
    model = TaggerFactory.load(PATH_TO_PRETRAINED + MODEL_NAME, -1)
    print (model.gpu)

bert_simple1.hdf5


/home/user/.local/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'src.layers.layer_context_word_embeddings_bert.LayerContextWordEmbeddingsBert' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/user/.local/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/user/.local/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the origin

-2


In [6]:
import tensorflow

In [8]:
def create_sequence_from_sentence(str_sentences):
    return [str_sentence.lower().split() for str_sentence in str_sentences]

In [9]:
words = create_sequence_from_sentence(['what is better amazon or itunes', 'who is better mouse or rat', 'what is easier to make bread or pizza', 'what is better for startap python or mathlab', 'what is better memory foam or gel memory foam', 'what is better perl or python', 'what is better cuda or opencl', 'what is better gamecube or ps2', 'what is preferable beer or milk'])

In [27]:
import sys
sys.path.append('/home/vika/NER_RNN/targer')
from src.factories.factory_tagger import TaggerFactory

def create_sequence_from_sentence(str_sentences):
    return [str_sentence.lower().split() for str_sentence in str_sentences]

class extractor:
    def __init__(self, input_sentence, model_name = 'bert_simple1.hdf5', model_path = '/home/vika/cqas_flask/external_pretrained_models/'):
        self.input_str = input_sentence
        self.answ = "UNKNOWN ERROR"
        self.model_name = model_name
        self.model_path = model_path
        self.first_object = ''
        self.second_object = ''
        self.predicates = ''
        
    def get_objects_predicates(self, list_words, list_tags):
        obj_list = []
        pred_list = []
        for ind, elem in enumerate(list_tags):
            if elem == 'B-OBJ':
                obj_list.append(list_words[ind])
            if elem == 'B-PREDFULL':
                pred_list.append(list_words[ind])    
        return obj_list, pred_list
    
    def extract_objects_predicates(self, input_sentence):
        words = create_sequence_from_sentence([input_sentence])
        print (words)
        model = TaggerFactory.load(self.model_path + self.model_name, -1)
        print (model.gpu)
        #model.cuda(device=2)
        #model.gpu = 2
        tags = model.predict_tags_from_words(words)
        print (tags)
        objects, predicates = self.get_objects_predicates(words[0], tags[0])
        print (objects)
        print (predicates)
        self.predicates = predicates
        if len(objects) >= 2:
            self.first_object = objects[0]
            self.second_object = objects[1]
        else:
            print("We have %d objects to compare %s" %(len(objects), objects))
            split_sent = words[0]
            if 'or' in split_sent:
                or_index = split_sent.index('or')
                print ("or split_sent", or_index)
                try:
                    obj1 = split_sent[or_index - 1]
                    obj2 = split_sent[or_index + 1]
                    print (obj1, obj2)
                    self.first_object = obj1
                    self.second_object = obj2
                except:
                    self.answ = "We can't recognize two objects for compare" 
            elif 'vs' in split_sent:
                or_index = split_sent.index('vs')
                try:
                    obj1 = split_sent[or_index - 1]
                    obj2 = split_sent[or_index + 1]
                    print (obj1, obj2)
                    self.first_object = obj1
                    self.second_object = obj2
                except:
                    self.answ = "We can't recognize two objects for compare" 
            else:
                self.answ = "We can't recognize two objects for compare" 
    def get_params(self):
        self.extract_objects_predicates(self.input_str)
        return self.first_object, self.second_object, self.predicates

In [28]:
my_extractor = extractor("what is testier bread or pizza")
my_extractor.get_params()

[['what', 'is', 'testier', 'bread', 'or', 'pizza']]
-2
[['O', 'O', 'B-PREDFULL', 'B-OBJ', 'O', 'O']]
['bread']
['testier']
We have 1 objects to compare ['bread']
or split_sent 4
bread pizza


('bread', 'pizza', ['testier'])

In [29]:
import json
import requests

URL = 'http://ltdemos.informatik.uni-hamburg.de/cam-api'

proxies = {
  "http": "http://185.46.212.97:10015/",
  "https": "https://185.46.212.98:10015/",
}

def get_response(first_object, second_object, fast_search=True, 
               aspects=None, weights=None):
    num_aspects = len(aspects) if aspects is not None else 0
    num_weights = len(weights) if weights is not None else 0
    if num_aspects != num_weights:
        raise ValueError(
            "Number of weights should be equal to the number of aspects")
    params = {
        'objectA': first_object,
        'objectB': second_object,
        'fs': str(fast_search).lower()
    }
    if num_aspects:
        params.update({'aspect{}'.format(i + 1): aspect 
                       for i, aspect in enumerate(aspects)})
        params.update({'weight{}'.format(i + 1): weight 
                       for i, weight in enumerate(weights)})
    print ("get url")
    response = requests.get(url=URL, params=params, proxies=proxies)
    return response

In [30]:
class responser:
    def __init__(self):
        self.URL = 'http://ltdemos.informatik.uni-hamburg.de/cam-api'
        self.proxies = {"http": "http://185.46.212.97:10015/","https": "https://185.46.212.98:10015/",}
        
    def get_response(self, first_object, second_object, fast_search=True, 
               aspects=None, weights=None):
        print ("aspects", aspects)
        print ("weights", weights)
        num_aspects = len(aspects) if aspects is not None else 0
        num_weights = len(weights) if weights is not None else 0
        if num_aspects != num_weights:
            raise ValueError(
                "Number of weights should be equal to the number of aspects")
        params = {
            'objectA': first_object,
            'objectB': second_object,
            'fs': str(fast_search).lower()
        }
        if num_aspects:
            params.update({'aspect{}'.format(i + 1): aspect 
                           for i, aspect in enumerate(aspects)})
            params.update({'weight{}'.format(i + 1): weight 
                           for i, weight in enumerate(weights)})
        print ("get url")
        response = requests.get(url=URL, params=params, proxies=proxies)
        return response

In [26]:
URL = 'http://ltdemos.informatik.uni-hamburg.de/cam-api'
proxies = {"http": "http://185.46.212.97:10015/","https": "https://185.46.212.98:10015/",}
params = {
            'objectA': 'cat',
            'objectB': 'dog',
            'fs': str(True).lower()}
response = requests.get(url=URL, params=params, proxies=proxies)

In [23]:
import numpy as np
np.ones(5)

array([1., 1., 1., 1., 1.])

In [32]:
my_responser = responser()
obj1, obj2, predicates = my_extractor.get_params()
print (obj1, obj2, predicates)
if (len(obj1) > 0 and len(obj2) > 0):
    response = my_responser.get_response(first_object = obj1, second_object = obj2, fast_search=True, aspects = predicates, weights = [1 for predicate in predicates])
else:
    print ("smth wrong with objects obj1 %s obj2 %s", obj1, obj2)

[['what', 'is', 'testier', 'bread', 'or', 'pizza']]
-2
[['O', 'O', 'B-PREDFULL', 'B-OBJ', 'O', 'O']]
['bread']
['testier']
We have 1 objects to compare ['bread']
or split_sent 4
bread pizza
bread pizza ['testier']
aspects ['testier']
weights [1]
get url


In [33]:
b = response.json()

In [36]:
def answerer(input_string):
    my_extractor = extractor("what is testier bread or pizza")
    my_responser = responser()
    obj1, obj2, predicates = my_extractor.get_params()
    print (obj1, obj2, predicates)
    if (len(obj1) > 0 and len(obj2) > 0):
        response =  my_responser.get_response(first_object = obj1, second_object = obj2, fast_search=True, aspects = predicates, weights = [1 for predicate in predicates])
        try:
            response_json = response.json()
            my_diviner = diviner()
            my_diviner.create_from_json(response_json)
            answer = my_diviner.generate_advice()
            return answer
        except:
            return ("smth wrong in response, please try again")
    else:
        return ("smth wrong with objects obj1 %s obj2 %s", obj1, obj2)
        

In [37]:
answerer("what is tastier bread or pizza")

[['what', 'is', 'testier', 'bread', 'or', 'pizza']]
-2
[['O', 'O', 'B-PREDFULL', 'B-OBJ', 'O', 'O']]
['bread']
['testier']
We have 1 objects to compare ['bread']
or split_sent 4
bread pizza
bread pizza ['testier']
aspects ['testier']
weights [1]
get url


'smth wrong in response, please try again'

In [38]:
from generation.generation import diviner

response_json = response.json()
Merlin = diviner()
Merlin.create_from_json(response_json)
Merlin.generate_advice()

winnder: pizza  other: bread
acpect winner  faster, easier to make, lighter, bigger, simpler, greater, easier to handle, easier, easier to eat
acpect other  safer, quicker, easier to find and more expensive, easier for me to make, nicer


Pizza is better than bread, because it is faster, easier to make, lighter, bigger, simpler, greater, easier to handle, easier, easier to eat. 
 At the same time, bread is safer, quicker, easier to find and more expensive, easier for me to make, nicer


In [39]:
from my_functions import do_sum, answerer
answerer("what is better bread o pizza")

[['what', 'is', 'testier', 'bread', 'or', 'pizza']]
-2
[['O', 'O', 'B-PREDFULL', 'B-OBJ', 'O', 'O']]
['bread']
['testier']
We have 1 objects to compare ['bread']
sentense [['what', 'is', 'testier', 'bread', 'or', 'pizza']]
split sent ['what', 'is', 'testier', 'bread', 'or', 'pizza']
have or
4
obj1 bread
obj2 pizza
self.first_object bread
self.second_object pizza
obj1, obj2, predicates bread pizza ['testier']
aspects ['testier']
weights [1]
get url
winnder: pizza  other: bread
acpect winner  faster, easier to make, lighter, bigger, simpler, greater, easier to handle, easier, easier to eat
acpect other  safer, quicker, easier to find and more expensive, easier for me to make, nicer


Pizza is better than bread, because it is faster, easier to make, lighter, bigger, simpler, greater, easier to handle, easier, easier to eat. 
 At the same time, bread is safer, quicker, easier to find and more expensive, easier for me to make, nicer
